# 导入模块

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [ ]:
# 黑名单
sql_black = '''
select cause, count(id_card) from db_credit.blacklist group by cause order by count(id_card) desc
'''
df_black = clean.query(sql_black)
# df_black.to_excel('F:/需求/总需求/黑名单.xlsx', index=False)

In [ ]:
t_date = datetime.now().strftime('%Y%m%d')

In [ ]:
# df_order = pd.read_excel('F:/需求/七月需求/订单明细.xlsx')
df_order = pd.read_excel('F:/需求/七月需求/无标题(4).xlsx')
# df_order = pd.read_excel('F:/需求/七月需求/无标题(3).xlsx')
df_order.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'status':'订单状态', 'total_freeze_fund_amount': '冻结只有资金'}, inplace=True)
df_order['下单日期']=df_order['下单时间'].dt.date
df_order


In [ ]:
df_order['全免押'] = np.where(df_order['冻结只有资金']==0,1,0)
df_order['全不免押'] = np.where(df_order['冻结只有资金']==-1,1,0)
df_order['部分免押'] = np.where((df_order['冻结只有资金']!=-1)&(df_order['冻结只有资金']!=0),1,0)
df_order

In [ ]:
# 快捷下单
df_kj = df_order[df_order['order_method']==1]
df_kj_group = df_kj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})
# df_kj

In [ ]:
# 非快捷下单
df_fkj = df_order[df_order['order_method']==0]
df_fkj_group = df_fkj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# iPhone16系列
df_order['name'] = df_order['name'].fillna('未知')
df_16 = df_order[df_order['name'].str.contains(pat='iPhone16', regex=True)]
df_16_group = df_16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# 非iPhone16系列
df_f16 = df_order[~df_order['name'].str.contains(pat='iPhone16', regex=True)]
df_f16_group = df_f16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [ ]:
# Today1 = datetime.now().strftime('%Y%m%d%H')
# with pd.ExcelWriter(f'F:/需求/七月需求/订单明细汇总_机型_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_16_group.to_excel(writer, sheet_name='iPhone16')
#     df_f16_group.to_excel(writer, sheet_name='非iPhone16')

In [ ]:
# with pd.ExcelWriter(f'F:/需求/七月需求/订单明细汇总_快捷下单_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_kj_group.to_excel(writer, sheet_name='快捷下单')
#     df_fkj_group.to_excel(writer, sheet_name='非快捷下单')

In [ ]:
# df_data = pd.read_excel('F:/日报/data.xlsx')
# df_data['日期'] = pd.to_datetime(df_data['日期'])
# df_data = df_data.set_index('日期')
# df_data = df_data[['去重订单', '前置拦截', '进件', '机审强拒', '电审拒绝', '客户取消', '无法联系', '出库前风控强拒']]

In [ ]:
# weekly_sum = W_group(custom_weekly_resampler(df_data)).drop(columns=['week_day'])
# group(weekly_sum)[['week_group', '去重订单','前置拦截','前置拦截率','进件','进件通过率','机审强拒','机审强拒比例','电审拒绝','电审拒绝比例','客户取消','客户取消率','无法联系','无法联系比例','出库前风控强拒','出库前风控强拒比例']]

In [ ]:
# df_ck3 = pd.read_excel('F:/租后表分析数据_ld/出库单三方数据表现-1114.xlsx')


In [ ]:
sql_3 = '''
SELECT om.order_number, tor.overdue_day
FROM db_digua_business.t_overdue_record tor 
left join db_digua_business.t_order om on tor.order_id=om.id
'''
df_3 = clean.query(sql_3)

In [ ]:
# df_ck31 = df_ck3[['order_number', 'new_overdue_day']].drop_duplicates()
# df_31 = df_3.drop_duplicates()
# df_ck31
# df_ck3113 = pd.merge(df_31, df_ck31, on='order_number', how='right')
# df_ck3113[(df_ck3113['overdue_day'].notna())&(df_ck3113['new_overdue_day'].notna())&(df_ck3113['overdue_day']!=df_ck3113['new_overdue_day'])]

# 截止到当前时间首逾

## 查询订单

In [ ]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金new_actual_money，实收租金（已收）rent_received，未收租金new_actual_money-rent_received，分配人
sql = '''
select 
om.create_time, om.order_number, tprm.order_id, tprm.rent_received 实收租金
, ymos.refund_date, ymos.reality_refund_date
, om.status, tod.new_actual_money 应收租金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 资方订单分期表
left join (select order_id, refund_date, money, reality_refund_date from db_rent.ya_merchant_order_stages where sort=2) ymos on ymos.order_id=om.id
where date_format(om.create_time, '%Y-%m-%d')>='2025-05-12'
and date_format(om.create_time, '%Y-%m-%d')<='2025-06-12'  -- 设置时间范围
'''
df = clean.query(sql)
df

## 逾期天数

In [ ]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金，实收租金，未收租金，分配人
# status=4:租赁中
t_date = datetime.now().date()
df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df.loc[:, '下单日期'] = pd.to_datetime(df.create_time.dt.date)
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
# 首逾：sort=2，refund_date<now_day|reality_refund_date>refund_date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
df.loc[:, '未收租金'] = df.应收租金-df.实收租金
df_new = df[(df.overdue_day>0)&(df.reality_refund_date.isna())&(df.status==4)]
df_new

## 分配人

In [ ]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = clean.query(sql_name)
df_name


In [ ]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['李巧玲','李巧凤','刘三妹','何静','谢金凤','廖丽敏','李楠','黄兰娟','杨健','林思慧','胡彩滢','周莹','罗芳', '周念慈', '周汉鸿']
# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# df_name = df_name[df_name.信审员.isin(name_list)]
df_name

In [ ]:
# 关联分配人
df_all = df_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
df_all

# 六月份买断

## 查询订单

In [ ]:
# 统计六月份买断（买断表t_order_out，买断时间pay_date筛选）的订单数、使用了多少买断优惠劵、优惠前买断金额、优惠后买断金额、采购金额、毛利率（优惠前/后买断金额-采购金额）/采购金额
# ybt.`status`'任务状态，0为待处理，1为跟进中，2已挂起，3已完成，4已取消'，too.status=2为已买断
sql_md= f'''
select
too.order_id 订单ID, tod.new_actual_money 优惠前买断金额, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 优惠后买断金额,
tprm.purchase_amount 采购金额, om.has_actual, date_format(too.pay_date, '%Y-%m') 月份
from db_digua_business.t_order_out too
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=too.order_id
where date_format(too.pay_date, '%Y-%m')='2025-06'
and too.status=2
'''
df_md = clean.query(sql_md)
df_md

In [ ]:

df_md.loc[:, '实际买断订单数'] = np.where(df_md.has_actual == 1, 1, 0)
# df_md.loc[:, '使用优惠劵订单数'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1) & (df_md.discount_money > 0), 1, 0)
df_md.loc[:, '使用优惠劵订单数'] = np.where(df_md.优惠金额 > 0, 1, 0)
# df_md.loc[:, '优惠后买断金额'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1),df_md.优惠后买断金额, 0)
df_md.groupby('月份').agg({'订单ID': 'count','使用优惠劵订单数': 'sum', '优惠前买断金额': 'sum', '优惠后买断金额': 'sum', '采购金额': 'sum'}).rename(columns={'订单ID': '买断订单数'})
df_md.loc[:, '发放优惠前的毛利率'] = ((df_md.优惠前买断金额 - df_md.采购金额) / df_md.采购金额).map(
        lambda x: format(x, '.2%'))
df_md.loc[:, '发放优惠后的毛利率'] = ((df_md.优惠后买断金额 - df_md.采购金额) / df_md.采购金额).map(
    lambda x: format(x, '.2%'))
# df_md
    

# 查询手机号对应的订单

In [ ]:
# 查询excel文件，sheet2页面中的手机号列，与数据库订单表中7月份订单手机号列进行匹配，并返回匹配结果的订单号和手机号
df_phone_number = pd.read_excel('D:/需求/进件记录.xlsx', sheet_name='Sheet2')
# 转str类型
df_phone_number['手机号'] = df_phone_number['手机号'].astype(str)
# 取出第一列数据去重，并转换为列表
# phone_numbers = df_phone_number.iloc[:, 0].drop_duplicates().tolist()
# phone_numbers
phone_number_list = tuple(df_phone_number.iloc[:, 0].drop_duplicates().to_list())
phone_number_list


In [ ]:
df = df[df.user_mobile.str.contains(r'\d+')]
# 在df中筛选出手机号在phone_numbers列表中的行
df_filtered = df[df['user_mobile'].isin(phone_numbers)]
# 返回匹配结果的订单号和手机号
df_ans = df_filtered[['order_number', 'user_mobile']]
df_ans



In [ ]:

# 查询订单
sql = f'''
select order_number,user_mobile from db_digua_business.t_order where create_time between '2023-07-01' and '2023-07-15' and user_mobile in {phone_number_list}
'''
df = query(sql)
df


In [ ]:
sql = '''
        SELECT om.order_number,tprm.overdue_type,ymos.reality_refund_date,ymos.refund_date

        FROM db_digua_business.t_postlease_receivables_monitoring  tprm
        LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
        LEFT JOIN db_rent.ya_merchant_order_stages ymos on om.id = ymos.order_id
        where ymos.refund_date >= DATE_ADD(CURRENT_DATE,INTERVAL +0 day )
        and ymos.refund_date <= DATE_ADD(CURRENT_DATE,INTERVAL +3 day )

        '''
        
df = clean.query(sql)

In [ ]:
# from datetime import datetime, timedelta
# # 读取excel文件清单
# # 跑出清单：
# # 1.当前无逾期
# # 2.当期未还款
# # 3.还款日为观察点及后3天，比如8.1跑的，那么还款日为8.1-8.4的同时满足 1和2的清单
# df_list = pd.read_excel('F:/需求/倩姐需求/zzyj_list.xlsx')

# # 拼接df_list与df,以订单号进行匹配
# df_list_merge = df_list.merge(df, on='order_number', how='inner')
# # 选择按照reality_refund_date降序排序，取第一条数据
# # df_list_merge = df_list_merge.sort_values(by='reality_refund_date', ascending=False).groupby('order_number').head(1)
# # # 获取当前无逾期的清单
# # # df_list_merge[df_list_merge.overdue_type.notna()].head()
# # df_list_merge_no_overdue = df_list_merge[df_list_merge.overdue_type=='']
# # # 获取当前未还款的清单
# # df_list_merge_no_repay = df_list_merge[df_list_merge.实付日期new.isna()]
# df_list_merge['实付日期new'] = pd.to_datetime(df_list_merge['实付日期new'])
# df_list_merge['实付日期new'] = np.where(df_list_merge['reality_refund_date'].notna(), df_list_merge['reality_refund_date'], df_list_merge['实付日期new'])

# # 获取今天的日期
# today = pd.Timestamp(datetime.now().date())
# # 获取当前需要还款的清单,应付日期为今天到后3天的范围，还款日为观察点及后3天，比如8.1跑的，那么还款日为8.1-8.4的同时满足 1和2的清单
# df_list_merge_repay = df_list_merge[(df_list_merge.应付日期 >= today) & (df_list_merge.应付日期 <= today + timedelta(days=3))& (df_list_merge.overdue_type=='') & (df_list_merge.实付日期new.isna()) ]
# df_list_merge_repay = df_list_merge_repay.drop_duplicates(subset=['order_number'])

In [ ]:
# df_list_merge[df_list_merge['order_number']=='A20250428224312197']

# 二手汇总

In [ ]:
excel_file = pd.ExcelFile('F:/需求/七月需求/24年1月-10月机型分析.xlsx')
# 从 excel_file 中解析名为 '1月' 的工作表
df = excel_file.parse('1月')[:-1]
for sheet in excel_file.sheet_names[1:]:
    df = pd.concat([df,excel_file.parse(sheet)[:-1]])
df

In [ ]:

# df['型号'] = df['型号'].str.replace(' ', '', regex=False)
# df
# df_group = df.groupby('型号').agg({'数量': 'sum', '总回收价格':'sum', '总采购金额': 'sum', '总官网金额': 'sum'})
# df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
# df_group['总采购金额均值'] = df_group['总采购金额']/df_group['数量']
# df_group['总官网金额均值'] = df_group['总官网金额']/df_group['数量']
# df_group = df_group.reset_index()
# df_group['型号_new'] = df_group['型号'].str.split('/').str[0]
# df_group['内存'] = df_group['型号'].str.split('/').str[1]
# df_group.sort_values(['数量'], ascending=[False]).to_excel('F:/需求/七月需求/24年1月-10月机型分析汇总.xlsx')

# 惜仪需求

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/二手机型.xlsx')


df_es['月份'] = df_es['入库日期'].dt.month
df_es['型号内存'] = df_es['型号_new'] + '_' + df_es['内存'] + '_' + df_es['颜色']

In [ ]:
month_list = df_es['月份'].drop_duplicates().to_list()
# 使用 groupby 方法按 '型号内存' 对数据进行分组，然后使用 agg 方法对每组数据进行聚合，计算 '颜色' 的数量、'回收价格'、'采购价' 和 '官网价' 的总和。
for ml in month_list:
    df_month = df_es[df_es['月份']==ml]
    df_group = df_month.groupby('型号内存').agg({'颜色': 'count', '回收价格': 'sum', '采购价':'sum', '官网价':'sum'})
    df_group.rename(columns={'颜色': '数量', '回收价格': '总回收价格', '采购价': '总采购价', '官网价': '总官网价'}, inplace=True)
    df_group.loc['合计'] = {'数量': df_group['数量'].sum(), '总回收价格': df_group['总回收价格'].sum(), '总采购价': df_group['总采购价'].sum(),
                                        '总官网价': df_group['总官网价'].sum()}
    df_group['总采购价比值'] = df_group['总回收价格']/df_group['总采购价']
    df_group['总采购价比值'] = df_group['总采购价比值'].apply(lambda x: format(x, '.2%'))

    df_group['总官网价比值'] = df_group['总回收价格']/df_group['总官网价']
    df_group['总官网价比值'] = df_group['总官网价比值'].apply(lambda x: format(x, '.2%'))
    # if ml==1:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')
    # else:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl', mode='a') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')

In [ ]:
sheets = pd.ExcelFile('F:/需求/二手手机/二手机型（月份）.xlsx')
df = sheets.parse('9月')[:-1]
for sheet in sheets.sheet_names[9:]:
    df = pd.concat([df,sheets.parse(sheet)[:-1]])
df


In [ ]:
def model_price(df):
    df_group = df.groupby('型号内存').agg({'数量': 'sum', '总回收价格':'sum', '总采购价': 'sum', '总官网价': 'sum'})
    df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
    df_group['总采购价均值'] = df_group['总采购价']/df_group['数量']
    df_group['总官网价均值'] = df_group['总官网价']/df_group['数量']
    df_group = df_group.reset_index()
    df_group['型号'] = df_group['型号内存'].str.split('_').str[0]
    df_group['内存'] = df_group['型号内存'].str.split('_').str[1]
    df_group['颜色'] = df_group['型号内存'].str.split('_').str[2]
    return df_group

In [ ]:
df['型号内存'] = df['型号内存'].str.replace(' ', '', regex=False)
df = df[df['型号内存'].str.contains((r'iPhone14|iPhone15'))]
df = df[df['型号内存'].str.contains((r'99新|95新|准新|90新'))]
df_ds = df[df['型号内存'].str.contains((r'带锁|有锁'))]
df_wds = df[~df['型号内存'].str.contains((r'带锁|有锁'))]
# model_price(df_ds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_带锁.xlsx')
# model_price(df_wds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_未带锁.xlsx')
# df_group.sort_values(['数量'], ascending=[False]).to_excel(f'F:/需求/七月需求/24年1月-10月机型分析汇总_{t_date}.xlsx')
# df_group.sort_values(['数量'], ascending=[False])#.to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}.xlsx')

# 获取优惠5折数据

In [ ]:
df_5 = pd.read_excel('F:/需求/瑞彤需求/总赠送5折优惠卷.xlsx')
df_5['订单号'] = df_5['订单号'].apply(lambda x: x.split('\\')[0].strip())
order_list = tuple(df_5['订单号'].to_list())
df_5['订单号'].shape

In [ ]:
sql=f'''
SELECT tprm.order_number 订单号, tprm.contract_price 买断价, tprm.purchase_amount 采购价,  tos.money 月租金, (tprm.contract_price-tprm.all_rental) 买断尾款
, tprm.all_deposit 商品押金, tprm.all_rental 总租金, tod.product_name 产品名称
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_stages tos on tos.order_id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
WHERE tprm.order_number in {order_list} and tos.sort=2
'''
df = query(sql)

In [ ]:
# df.to_excel('F:/需求/瑞彤需求/总赠送5折优惠卷_new.xlsx', index=False)

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/22年-23年二手机型.xlsx')
df_es = df_es[df_es['订单号'].str.contains('A2023')]


In [ ]:
df_es_group = df_es.groupby(['型号', '内存']).agg({'订单号': 'count', '售价': 'sum', '采购成本': 'sum', '官网价': 'sum'})
df_es_group['回收价'] = df_es_group['售价'] / df_es_group['订单号']
df_es_group['采购价'] = df_es_group['采购成本'] / df_es_group['订单号']
df_es_group['官网价均值'] = df_es_group['官网价'] / df_es_group['订单号']
# df_es_group

# 商家

In [ ]:
# 黑名单
sql4 = '''
SELECT
id_card, mobile
FROM db_credit.blacklist b
WHERE b.end_time is NULL or b.end_time>SYSDATE() -- 结束时间为空且大于当前时间，则为黑名单
'''
df4 = query(sql4)

In [ ]:
# 处理mobile字段并修改字段类型
df4 = df4[df4.mobile.str.contains(r'\d+')]
df4['id_card'] = df4['id_card'].astype(str)
df4['mobile'] = df4['mobile'].astype(int)

In [ ]:
# 导入Excel文件
df_customer = pd.ExcelFile('F:/需求/总需求/商家/客户名单.xlsx')
df_customer1 = df_customer.parse(sheet_name='三要素')
df_customer2 = df_customer.parse(sheet_name='二要素')

In [ ]:
df_customer1['now_date'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
df_customer1

In [ ]:
# 三要素 匹配身份证号和手机号
df_customer1['身份证'] = df_customer1['身份证'].astype(str)
df_customer_list = df_customer1.身份证.to_list()
df4[df4.id_card.isin(df_customer_list)]
# 匹配手机号
df_customer_mobile_list = df_customer1.手机号.to_list()
df4[df4.mobile.isin(df_customer_mobile_list)]

In [ ]:
# 匹配手机号
df_customer_mobile_list2 = df_customer2.mobile.to_list()
df4[df4.mobile.isin(df_customer_mobile_list2)]

## 年龄分布

In [ ]:
# 定义一个函数来计算年龄
def calculate_age(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 应用函数到 DataFrame
df_customer1['age'] = df_customer1.apply(lambda row: calculate_age(row['身份证'], row['now_date']), axis=1)


In [ ]:
# 定义年龄区间
age_bins = [-999, 18, 20, 23, 25, 30, 35, 40, 45, float('inf')]
age_labels = ['[-999,18)','[18,20)',  '[20,23)', '[23,25)', '[25,30)', '[30,35)', '[35,40)', '[40,45)', '[45+)']

# 使用 pd.cut 设置年龄区间
df_customer1['age_group'] = pd.cut(df_customer1['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=False)


In [ ]:
df_customer1[df_customer1.age<=26].shape

In [ ]:
# 三要素
df_age = df_customer1.groupby('age_group').agg(数量=('身份证', 'count'))
df_age.to_excel('F:/需求/总需求/商家/年龄分布1.xlsx')

# 交易流水汇总

In [ ]:
df_ls = pd.read_excel('F:/需求/七月需求/无标题(14).xlsx')

In [ ]:
df_ls.loc[:, '交易发起日期'] = df_ls.交易发起时间.dt.date

In [ ]:
# df_ls.groupby('交易发起日期').agg(总应付金额=('应付金额', 'sum'), 总实付金额=('实付金额（如果为空则应付=实付）', 'sum')).to_excel('F:/需求/七月需求/交易流水汇总.xlsx')

In [ ]:
df_ls2 = pd.read_excel('F:/需求/七月需求/无标题(15).xlsx')

In [ ]:
def pay_way(way):
    if way == 'WX':
        return '微信'
    elif way == 'WXXCX':
        return  "微信小程序"
    elif way == 'ZFB':
        return '支付宝'
    elif way == 'ZFBYSQ':
        return '支付宝预授权'
    elif way == 'ZFBXCX':
        return '支付宝小程序'
    elif way == 'DYXCX':
        return '抖音小程序'
    elif way == 'DY_HX_PAY':
        return '抖音小程序核销'
    elif way == 'YE':
        return '余额'
    elif way == 'WXZFF':
        return '微信支付分'
    elif way == 'ZFBZQQ':
        return '支付宝周期扣款'
    elif way == 'XX':
        return '线下'
    elif way == 'UNIONXX':
        return '银联线下'
    elif way == 'DEPOSIT':
        return '押金抵扣'
    elif way == 'RY_PAY':
        return '第三方代扣'
    elif way == 'ANT_PAY':
        return '蚂蚁链代扣'
    elif way == 'ZYX_PAY':
        return '智赢行支付'
    elif way == 'ZFB_WAP_PAY':
        return '支付宝h5'
    elif way == 'ZFBYWD':
        return '支付宝业务单代扣'
    elif way == 'ZFXXSK':
        return '线下（对资方）支付'
    elif way == 'XXX':
        return '线下（对维客）支付'
    elif way == 'XX_MERCHANT':
        return '线下（对商家）支付'
    elif way == 'UNICOM_PAY':
        return '联通号卡支付'

In [ ]:
df_ls2.loc[:, '支付方式_中文'] = df_ls2.apply(lambda x: pay_way(x['支付方式']), axis=1)
# df_ls2.to_excel('F:/需求/七月需求/支付宝交易流水.xlsx')

In [ ]:
df_ls2.loc[:, '交易发起日期'] = df_ls2.交易发起时间.dt.date
df_ls2_3 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权'])]
df_ls2_3_group = df_ls2_3.groupby('交易发起日期').agg(总应付金额_前三=('应付', 'sum'), 总实付金额_前三=('实付', 'sum'))
df_ls2_4 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权', '智赢行支付'])]
df_ls2_4_group = df_ls2_4.groupby('交易发起日期').agg(总应付金额_后四=('应付', 'sum'), 总实付金额_后四=('实付', 'sum'))
# df_ls2_group = df_ls2_3_group.merge(df_ls2_4_group, on='交易发起日期', how='inner').to_excel('F:/需求/七月需求/支付方式交易汇总.xlsx')

In [ ]:
df_ls2_3_group


# 匹配云商分

In [ ]:
# trace关联
sql_trace='''-- trace关联表 
SELECT order_trace_id, trace_id, order_id, order_number, risk_trace_id
from db_credit.order_association
;
'''
trace_df = query(sql_trace)
# 云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT trace_id, query_rs
from db_credit.risk_bairong_scoreysstd
;
'''
br_ys_df =query(sql1)
br_ys_df['云商分'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
br_ys_df['云商分'] = br_ys_df['云商分'].astype(int)
df_rule = trace_df.merge(br_ys_df[['trace_id', '云商分']], on='trace_id', how='outer')


In [ ]:
df_y = pd.read_excel('F:/租后表分析数据_ld/tmp(1).xlsx')

In [ ]:
# df_y[['order_id', 'order_number', '拒绝理由']].merge(df_rule[['order_id', '云商分']], on='order_id', how='left').to_excel('F:/需求/总需求/强拒云商分.xlsx', index=False)

In [ ]:
# （26）年龄、性别、总租金
# 定义一个函数来计算年龄
def calculate_age1(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 定义一个函数来判断性别
def get_gender(id_card):
    gender_digit = int(id_card[-2])  # 第17位数字
    return '男' if gender_digit % 2 != 0 else '女'

# 取基本信息
sql = '''
SELECT  om.order_number, om.create_time
,om.all_money as all_rental
,tmu.id_card_num
,tod.actual_money

from db_digua_business.t_order om
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
-- where date_format(om.create_time, '%Y-%m-%d') >= '2024-11-01'
;
'''

df_zhys = query(sql)

# df_zhys.dtypes
df_zhys['下单日期'] = df_zhys['create_time'].dt.date
df_zhys['order_number'] = df_zhys['order_number'].astype(str)
# 取样本
path = 'F:/租后表分析数据_ld/'
data_model = pd.read_excel(path + 'tmp(2).xlsx')
data_model['order_number'] = data_model['order_number'].astype(str)
# data_model


In [ ]:
# 关联
data_model = data_model.merge(df_zhys, on = 'order_number', how = 'left')
data_model.shape
data_model[:2]

In [ ]:

data_model1 = data_model[data_model.id_card_num.isna()]
data_model2 = data_model[data_model.id_card_num.notna()]
# data_model = data_model.apply(lambda x: get_gender(x.id_card_num), axis = 1)
data_model2['gender'] = data_model2['id_card_num'].apply(get_gender)
data_model2['age'] = data_model2.apply(lambda row: calculate_age1(row['id_card_num'], row['下单日期']), axis=1)

data_model2[:2]

data_model3 = pd.concat([data_model2, data_model1])
data_model3.to_excel('F:/需求/总需求/性别年龄.xlsx', index=False)

# 拒绝理由

In [ ]:
df_reject = pd.read_excel('F:/租后表分析数据_ld/riskdemo2.xlsx')
df_rejected = df_reject[df_reject.强拒结果==1]
# 前置风控
df_rejected_qz = df_rejected[df_rejected.前置风控.notna()]
df_rejected_qz.loc[:, 'rejected'] = df_rejected_qz.前置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_qz = df_rejected_qz[df_rejected_qz.rejected.astype(str)!='[]']
# 前置风控
df_rejected_hz = df_rejected[df_rejected.后置风控.notna()]
df_rejected_hz.loc[:, 'rejected'] = df_rejected_hz.后置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_hz = df_rejected_hz[df_rejected_hz.rejected.astype(str)!='[]']

# 出库前风控
df_rejected_ck = df_rejected[df_rejected.出库前风控.notna()]
df_rejected_ck.loc[:, 'rejected'] = df_rejected_ck.出库前风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_ck = df_rejected_ck[df_rejected_ck.rejected.astype(str)!='[]']
df_rejected_ck.rejected.drop_duplicates()
# df_concat = pd.concat([df_rejected_qz[['手机号', 'rejected']], df_rejected_hz[['手机号', 'rejected']], df_rejected_ck[['手机号', 'rejected']]])
# df_concat = df_concat.explode('rejected')
# pd.crosstab('2024-01-21', df_concat['rejected'], margins=True).to_excel('F:/需求/总需求/拒绝理由.xlsx')

In [ ]:

df_rejected_qz.rejected.drop_duplicates()

In [ ]:
df_rejected_hz.rejected.drop_duplicates()

# 仕程需求

In [ ]:
# df_fp = pd.read_excel('F:/无标题(2).xlsx')
# df_fp = df_fp[~df_fp.agent_user_name.isna()]
# df_fp.loc[:, 'sort'] = df_fp.sort.str.strip('[]')
# df_fp.loc[:, '还款'] = np.where(df_fp.status==3, 1, 0)
# df_fp.loc[:, '未还款'] = np.where(df_fp.status!=3, 1, 0)
# df_fp_g = df_fp.groupby(['agent_user_name', 'sort']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_fp_g = df_fp_g.rename(columns={'agent_user_name': '名称', 'sort': '期数', 'order_id': '总任务'})
# df_fp_g.loc[:, '催回率'] = (df_fp_g.还款/df_fp_g.总任务).map(lambda x: format(x, '.2%'))
# # with pd.ExcelWriter('F:/需求/总需求/还款情况.xlsx', engine='xlsxwriter') as writer:
#     # df_fp_g.to_excel(writer, sheet_name='还款情况', index=False)


In [ ]:
# df_6month = pd.read_excel('F:/需求/仕程需求/6月起失联名单.xlsx')
# order_list = tuple(df_6month.订单号.to_list())
# sql_6month = f'''
# select
# om.order_number, ymos.refund_date, ymos.reality_refund_date, ymos.sort, tprm.rembursement_status, om.status
# from db_digua_business.t_order om 
# left join db_rent.ya_merchant_order_stages ymos on om.id=ymos.order_id
# left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
# where om.order_number in {order_list}
# '''
# df_month = query(sql_6month)
# now_day = datetime.now().strftime('%Y-%m-%d')
# df_month = df_month[((df_month.refund_date<now_day)&(df_month.reality_refund_date.isna()))|((df_month.rembursement_status=='已逾期')&(df_month.status==5))]
# df_month.drop_duplicates(subset=['order_number'], keep='last', inplace=True)
# month_list = df_month.order_number.to_list()
# df_6month.loc[:, '是否逾期'] = np.where(df_6month.订单号.isin(month_list), 1, 0)
# df_6month = df_6month.merge(df_month[['sort', 'order_number']], left_on='订单号', right_on='order_number', how='left')
# df_6month.iloc[:, :-1].to_excel('F:/需求/仕程需求/6月起失联名单_new.xlsx', index=False)

# 力哥

In [ ]:
# df  = pd.read_excel('F:/需求/力哥需求/4月份呼入.xlsx', sheet_name='Sheet1')
# df.loc[:, '分钟'] = df.通话时长.str[3:5]
# df.loc[:, '秒'] = df.通话时长.str[-2:]
# df.to_excel('F:/需求/力哥需求/4月份呼入_新.xlsx', index=False)

# 资方数据

In [ ]:
sql_zf = '''
SELECT tprm.order_number, om.merchant_name,tod.sku_attributes, tso.price as 碎屏险价格,
tod.new_actual_money '应收金额', tprm.purchase_amount '采购成本', (tprm.rent_received) '已收租金', tprm.overdue_type, top.residue_money '押金'
,case when ym.`name`='维客自营账户' then '维客自营账户' 
			when ym.`name`='海南鸿丰商业保理有限公司' then '海南鸿丰商业保理有限公司'
			when ym.`name`='成都越蓉金科科技有限公司' then '成都越蓉金科科技有限公司'
			when ym.`name`='杭州金锤网络信息服务有限公司' then '杭州金锤网络信息服务有限公司'
			else '其他' end '资方'
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT order_id, residue_money from db_digua_business.t_order_pledge where residue_money>=0) top on top.order_id=om.id
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id
where om.`status`=4
'''
df_zf = query(sql_zf)

In [ ]:
df_zf = df_zf[df_zf.采购成本>0]
df_zf = clean.drop_merchant(df_zf)
df_zf = clean.drop_rejected_merchant(df_zf)
df_zf.loc[:, '押金'] = df_zf.押金.fillna(0)
df_zf.loc[:, '已收租金'] = df_zf.已收租金+df_zf.押金
df_zf.loc[:, '应收未收金额'] = df_zf.应收金额-df_zf.已收租金
df_zf.loc[:, '应收未收采购成本'] = df_zf.采购成本-df_zf.已收租金
df_zf.loc[:, '应收未收中未逾期的金额'] = np.where(df_zf.overdue_type=='', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD1-30的金额'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD31-60的金额'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD61+的金额'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中未逾期的采购成本'] = np.where(df_zf.overdue_type=='', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD1-30的采购成本'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD31-60的采购成本'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD61+的采购成本'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收采购成本, 0)
df_zf_group = df_zf.groupby('资方').agg({'应收金额': 'sum', '应收未收金额': 'sum', '应收未收中未逾期的金额': 'sum', '应收未收中DPD1-30的金额': 'sum', '应收未收中DPD31-60的金额': 'sum', 
                            '应收未收中DPD61+的金额': 'sum', '采购成本': 'sum', '应收未收采购成本': 'sum', '应收未收中未逾期的采购成本': 'sum', '应收未收中DPD1-30的采购成本' : 'sum',
                            '应收未收中DPD31-60的采购成本': 'sum', '应收未收中DPD61+的采购成本': 'sum', '碎屏险价格': 'sum'})
df_zf_group.sort_values('资方', ascending=False).to_excel('F:/需求/总需求/资方数据.xlsx')

In [ ]:
df_zf[(df_zf.应收未收中未逾期的采购成本 >= 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()
df_zf[(df_zf.应收未收中未逾期的采购成本 < 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()

## 资方数据鸿丰

In [3]:
sql_zf2 = '''
select
om.id, om.status, ym.name, om.create_time
from db_digua_business.t_order om
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
where ym.name in ('海南鸿丰商业保理有限公司', '杭州金锤网络信息服务有限公司')
'''
df_zf2 = clean.query(sql_zf2)

In [4]:
df_zf2.loc[:, '下单月份'] = df_zf2.create_time.dt.strftime('%Y-%m')
df_zf2.loc[:, '进件'] = np.where(df_zf2.status.isin([1,13]), 0, 1)
df_zf2.loc[:, '出库'] = np.where(df_zf2.status.isin([2,3,4,5,6,8,15]), 1, 0)
df_zf2_group = df_zf2.groupby('下单月份').agg({'进件': 'sum', '出库': 'sum'})
df_zf2_group.loc['合计', :] = df_zf2_group.sum(axis=0)
df_zf2_group#.to_excel('F:/需求/七月需求/资方数据_金锤和鸿丰.xlsx')

,进件,出库
下单月份,,
2024-08,11.0,7.0
2024-09,672.0,626.0
2024-10,239.0,231.0
2024-11,468.0,459.0
2024-12,779.0,740.0
2025-01,121.0,117.0
2025-02,88.0,84.0
2025-03,320.0,298.0
2025-04,401.0,392.0


# 财务数据

In [ ]:
def cw(month):
    sql_cw = f'''
    SELECT
    om.order_number 订单号, ym.name 名称, ymos.sort 期数, ymos.money 租金
    from db_digua_business.t_order om 
    left join db_rent.ya_merchant ym on om.merchant_id=ym.shop_id
    left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    where om.status=4
    and ym.shop_id in (14, 36, 148)
    and date_format(ymos.refund_date, '%Y-%m')='{month}'
    '''
    df_cw = query(sql_cw)
    df_cw_g = df_cw.groupby('名称').agg({'租金': 'sum'}).rename(columns={'租金': '应收租金'}).rename_axis('归属')
    df_cw_g.loc['合计',:] = df_cw_g.应收租金.sum()
    return df_cw, df_cw_g

# df_cw.to_excel('F:/需求/总需求/财务数据.xlsx', index=False)

In [ ]:
month = 5
# df_cw1, df_cw1_g = cw('2025-01')
# df_cw2, df_cw2_g = cw('2025-02')
# df_cw3, df_cw3_g = cw('2025-03')
df_cw5, df_cw5_g = cw('2025-05')
with pd.ExcelWriter(f'F:/需求/总需求/财务数据_{month}月.xlsx', engine='xlsxwriter') as writer:
    # df_cw1.to_excel(writer, sheet_name='一月份明细', index=False)
    # df_cw1_g.to_excel(writer, sheet_name='一月份汇总')
    # df_cw2.to_excel(writer, sheet_name='二月份明细', index=False)
    # df_cw2_g.to_excel(writer, sheet_name='二月份汇总')
    # df_cw3.to_excel(writer, sheet_name='三月份明细', index=False)
    # df_cw3_g.to_excel(writer, sheet_name='三月份汇总')
    df_cw5.to_excel(writer, sheet_name='五月份明细', index=False)
    df_cw5_g.to_excel(writer, sheet_name='五月份汇总')

In [ ]:
sql_qg = '''
select 
tprm.relet_periods,tod.new_actual_money,tprm.rent_received,
tprm.overdue_periods,tprm.total_receivable, tm.name
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_merchant tm on tm.id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
where tm.`name` in ('南宁微美汇', '广州捷瑞环保科技')
'''
df_qg = query(sql_qg)


In [ ]:
df_qg.loc[:, '续租剩余总未收'] = np.where(df_qg.relet_periods>0, df_qg.new_actual_money-df_qg.rent_received, 0)
df_qg.loc[:, '逾期总应收'] = np.where(df_qg.overdue_periods>0, df_qg.total_receivable, 0)
df_qg.groupby('name').agg({'续租剩余总未收': 'sum', '逾期总应收': 'sum'}).to_excel('F:/需求/奇哥需求/商家数据.xlsx')

# 廖志彧需求

In [ ]:
# 按结束时间买断订单数
sql_lzy = '''
select
ybt.order_id 订单ID, tod.new_actual_money 买断价, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 实收买断金额,
tprm.purchase_amount 采购金额, ybt.`status`, ybt.follow_log_tag, ybt.type, om.has_actual, date_format(ybt.end_time, '%Y-%m') 月份
from db_rent.ya_buyout_task ybt
left join (select order_id, discount_money from db_digua_business.t_order_out where status=2) too on too.order_id=ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=ybt.order_id
where date_format(ybt.end_time, '%Y-%m')='2025-05'
'''
# 提前买断订单对应期数
sql_stages = '''
select order_id 订单ID, sort 期数 from db_rent.ya_merchant_order_stages where date_format(refund_date, '%Y-%m')='2025-05'
'''
# 按发布时间买断订单数
sql_amd = '''
select ybt.order_id 发布时间订单号, tod.new_actual_money 发布时间买断价, date_format(ybt.add_time, '%Y-%m') 月份 from db_rent.ya_buyout_task ybt 
left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
where date_format(ybt.add_time, '%Y-%m')='2025-05'
'''
df_amd = query(sql_amd)

df_amd_g = df_amd.groupby('月份').agg({'发布时间订单号': 'count', '发布时间买断价': 'sum'})
df_lzy = query(sql_lzy)
df_stages = query(sql_stages)

# 发布买断订单数、买断订单数、发布买断金额（采购金额）、发布买断金额（买断价）、实收买断金额、实际买断订单数、买断优惠券总额、发放优惠前的毛利率、发放优惠后的毛利率

In [ ]:
df_lzy.loc[:, '实际买断订单数'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), 1, 0)
df_lzy.loc[:, '实收买断金额'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), df_lzy.实收买断金额, 0)
def md(df):
    df_g = df.groupby('月份').agg({'订单ID': 'count', '采购金额': 'sum', '买断价': 'sum', '实际买断订单数': 'sum', '实收买断金额': 'sum', '优惠金额': 'sum'}).rename(columns={'订单ID': '发布买断订单数'})
    df_g.loc[:, '优惠比例'] = (df_g.优惠金额/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠前的毛利率'] = ((df_g.买断价-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠后的毛利率'] = ((df_g.实收买断金额-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    return df_g
df_lzy_ts = md(df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag==4)&(df_lzy.has_actual==1)])
# '任务类型ybt.type，0：自动发布，1手动创建',
df_lzy_tq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==1)&(df_lzy.has_actual==1)]
df_lzy_tq2 = md(df_lzy_tq)
df_lzy_tq3 = df_lzy_tq.merge(df_stages, on='订单ID', how='left')
df_lzy_dq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==0)&(df_lzy.has_actual==1)]
# df_lzy_dq = md(df_lzy_dq)
# df_lzy_dq2 = df_lzy_dq.merge(df_amd_g, on='月份', how='left')
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月买断订单.xlsx', engine='xlsxwriter') as writer:
#     df_lzy_ts.to_excel(writer, sheet_name='特殊申请')
#     df_lzy_tq2.to_excel(writer, sheet_name='提前买断')
#     df_lzy_tq3.to_excel(writer, sheet_name='提前买断明细', index=False)
#     df_lzy_dq.to_excel(writer, sheet_name='到期买断')
# df_lzy.to_excel('F:/需求/廖志彧需求/5月买断订单.xlsx', index=False)&(df_lzy.has_actual==1)

## 优召

In [50]:
sql_yz = '''
select
om.id order_id, om.create_time 下单时间,turt.discount_status, turt.id_card_num,tdi.money tdi_money,om.order_number ,turt.create_time 优召时间,turt.complete_time
,tdi.use_type, tdi.type, tdi.create_time 优惠订单创建时间
, tprm.all_rental, tprm.all_deposit, tprm.purchase_amount, om.status, tod.new_actual_money, date_format(turt.complete_time, '%Y-%m') 月份
, tos.money 分期金额
from db_digua_business.t_user_recall_task turt 

left join db_digua_business.t_order om on om.order_number=turt.order_number
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_number=turt.order_number
left join db_digua_business.t_discount_item tdi on tdi.user_id=om.user_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_stages tos on tos.order_id=om.id
where date_format(turt.complete_time, '%Y-%m-%d')>='2025-08-01'
-- and date_format(turt.complete_time, '%Y-%m-%d')<'2025-08-01'

'''
# and date_format(om.create_time, '%Y-%m-%d') >= date_format(turt.create_time, '%Y-%m-%d')
# and date_format(om.create_time, '%Y-%m-%d') <= date_format(turt.complete_time, '%Y-%m-%d')
# and turt.order_number is not null
df_yz = clean.query(sql_yz)
df_yz.shape

(8244, 18)

In [29]:
df_yz_n = df_yz.sort_values('优惠订单创建时间', ascending=False).groupby('id_card_num').head(1)
df_yz_n.shape

(80, 18)

In [30]:
# 检查订单重复
df_yz_n = df_yz.drop_duplicates(subset=['order_id'])
df_yz_n.shape

(24, 18)

In [57]:
# 剔除status=10的订单
# df_yz = df_yz[df_yz.status!=10]
# df_yz = df_yz.drop_duplicates(subset=['id_card_num'])
# 按id_card_num分组创建时间降序排序取第一个
df_yz = df_yz.sort_values('优惠订单创建时间', ascending=False).groupby('id_card_num').head(1)
# df_yz = df_yz.drop_duplicates(subset=['id_card_num'])
df_yz["下单日期"]=df_yz["下单时间"].dt.date
df_yz["下单日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")
df_yz["优召日期"]=df_yz["优召时间"].dt.date
df_yz["优召日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")
df_yz["完成日期"]=df_yz["complete_time"].dt.date
df_yz["完成日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")

df_yz.loc[:, '进件订单数'] = np.where(((df_yz.status==1)|(df_yz.status==13))&((df_yz.下单日期>=df_yz.优召日期)&(df_yz.下单日期<=df_yz.完成日期)), 0, 1)
df_yz.loc[:, '出库订单数'] = np.where((df_yz.order_number.notna()), 1, 0)# (df_yz.status.isin([2, 3, 4, 5, 6, 8, 15]))&
# '优惠卷类型type：1、商品满减券；2、首期优惠券；3、商品延期券；4、商品买断券', 5、租中优惠券
# 仅考虑type=5的情况当前优惠券状态为Y，并且优惠券类型为商品满减券（use_type=1）优惠金额等于优惠券金额（tdi_money），优惠券类型为租中优惠券（use_type=2）时，优惠金额等于订单金额乘以（1-tdi_money/100）
# 
df_yz.loc[:, '优惠券金额'] = np.where(
    (df_yz.discount_status == 'Y') & (df_yz.use_type == 1) & (df_yz.type==5), df_yz.tdi_money,
    np.where((df_yz.discount_status == 'Y') & (df_yz.use_type == 2) & (df_yz.type==5), df_yz.分期金额 * (1 - df_yz.tdi_money / 100), 0)
)
df_yz.loc[:, '买断价'] = np.where(df_yz.出库订单数==1, df_yz.new_actual_money, 0)
df_yz.loc[:, '采购成本'] = np.where(df_yz.出库订单数==1, df_yz.purchase_amount, 0)
df_yz_g = df_yz.groupby('月份').agg({'进件订单数': 'sum', '出库订单数': 'sum', '优惠券金额': 'sum', '采购成本': 'sum', 'all_rental': 'mean', 'all_deposit': 'mean', '买断价': 'sum'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值'})
df_yz_g.loc[:, '租售比'] = (df_yz_g.总租金均值/df_yz_g.总押金均值).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠比例'] = (df_yz_g.优惠券金额/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠前毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠后毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本-df_yz_g.优惠券金额)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
# 发布优召订单数、进件订单数、出库订单数、优惠券金额、采购成本、租售比
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月优召订单.xlsx', engine='xlsxwriter') as writer:
#     df_yz_g.to_excel(writer, sheet_name='优召')
df_yz_g

,进件订单数,出库订单数,优惠券金额,采购成本,总租金均值,总押金均值,买断价,租售比,优惠比例,优惠前毛利率,优惠后毛利率
月份,,,,,,,,,,,
2025-07,80,23,3410.80,180998.0,7352.185652,9681.360870,278700.8,75.94%,1.88%,53.98%,52.10%
2025-08,172,52,5025.16,433408.0,7617.495000,9943.076923,654853.4,76.61%,1.16%,51.09%,49.93%


In [6]:
df_yz_g

,进件订单数,出库订单数,优惠券金额,采购成本,总租金均值,总押金均值,买断价,租售比,优惠比例,优惠前毛利率,优惠后毛利率
月份,,,,,,,,,,,
2025-07,80,23,4143.42,180998.0,7352.185652,9681.36087,278700.8,75.94%,2.29%,53.98%,51.69%


In [10]:
# 80	57	23	5481.13	180998.0	7352.185652	9681.36087	277083.59	75.94%	3.03%	53.09%	50.06%
df_yz[(df_yz.discount_status=='Y')&(df_yz.type.isin([1, 5]))].shape#25	

(16, 25)

In [11]:
df_yz[(df_yz.order_number.notna())].shape#.采购成本.sum()

(23, 25)

In [58]:
df_yz[(df_yz.优惠券金额>0)].type.value_counts()#320684200110287411

type
5.0    30
Name: count, dtype: int64

## 挽留

In [20]:
sql_wl = '''
select  
tort.order_id, tprm.purchase_amount, tod.new_actual_money, tdi.money tdi_money, tdi.use_type, om.status, 
date_format(tort.create_time, '%Y-%m') 月份, tort.task_status,tdi.create_time 优惠订单创建时间,
tos.money 分期金额
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_discount_item tdi on tdi.discount_id=tort.discount_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_stages tos on tos.id=tort.discount_stage_id
where tdi.type = 5
-- date_format(tort.create_time, '%Y-%m-%d')>='2025-08'
-- and  DATE_FORMAT(tort.create_time, '%Y-%m-%d')>='2025-06-01'
-- 仅有租中优惠劵
-- and tdi.type = 5
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(986, 10)

In [21]:

df_wl.月份.unique()

array(['2025-03', '2025-04', '2025-05'], dtype=object)

In [ ]:
# df_wl = df_wl.sort_values('优惠订单创建时间', ascending=False).groupby('order_id').head(1)
df_wl.loc[:, '挽留成功订单数'] = np.where(df_wl.task_status=='SUCCESS', 1, 0)
df_wl.loc[:, '出库'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# use_type:'优惠券类型：1、满减；2、折扣；
df_wl.loc[:, '折扣优惠券'] = np.where(df_wl.use_type==2, df_wl.tdi_money, 0)
df_wl.loc[:, '满减优惠券'] = np.where(df_wl.use_type==1, df_wl.分期金额 * (1 - df_wl.tdi_money / 100), 0)
df_wl.loc[:, '优惠券总金额'] = df_wl.loc[:, '折扣优惠券'] + df_wl.loc[:, '满减优惠券']
df_wl.loc[:, '采购金额'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.purchase_amount, 0)
df_wl.loc[:, '买断价'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.new_actual_money, 0)
df_wl_g = df_wl.groupby('月份').agg({'order_id': 'count', '挽留成功订单数': 'sum', '出库': 'sum', '采购金额': 'sum', '买断价': 'sum', '优惠券总金额': 'sum', '折扣优惠券': 'sum', '满减优惠券': 'sum'}).rename(columns={'order_id': '挽留发布订单数'})
df_wl_g.loc[:, '优惠比例'] = (df_wl_g.优惠券总金额/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '满减优惠券比例'] = (df_wl_g.满减优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '折扣优惠券比例'] = (df_wl_g.折扣优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠前的毛利率'] = ((df_wl_g.买断价-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠后的毛利率'] = ((df_wl_g.买断价-df_wl_g.优惠券总金额-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g
# 优惠券（买断优惠券、租中优惠券）总金额、挽留发布任务数、采购金额、买断价、优惠比例（优惠券/采购金额）、发放优惠前的毛利率、发放优惠后的毛利率
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月挽留订单.xlsx', engine='xlsxwriter') as writer:
#     df_wl_g.to_excel(writer, sheet_name='挽留')

In [289]:
# 检查订单重复
df_wl[df_wl.order_id.duplicated()]
# df_wl

,order_id,purchase_amount,new_actual_money,money,type,status,月份,task_status,优惠订单创建时间,挽留成功订单数,出库,买断优惠券,租中优惠券,优惠券总金额,采购金额,买断价


# 宏生需求

In [8]:
# 支付宝、京东订单的月份毛利
sql_hs = '''
select
tprm.order_create_time, tprm.order_id 订单号, tprm.all_rental 总租金, tprm.purchase_amount 采购金额, tod.commodity_premium 商品溢价费, tod.new_actual_money 买断金额, tod.product_name, tod.sku_attributes
,om.order_type
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order om on om.id=tprm.order_id
where tprm.purchase_amount>0
and date_format(tprm.order_create_time, '%Y-%m-%d')>='2025-05-01'
and date_format(tprm.order_create_time, '%Y-%m-%d')<='2025-08-31'
'''
df_hs = clean.query(sql_hs)
df_hs.loc[:, '下单月份'] = df_hs.order_create_time.dt.strftime('%Y-%m')

In [9]:
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["采购金额"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["采购金额"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["采购金额"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["采购金额"]*0.375
    else:
        return s["采购金额"]*0.425
df_hs.loc[:, '租赁方案'] = np.where(df_hs["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_hs.loc[:, '残值机型'] = np.where(df_hs["product_name"].str.contains('iPhone'),  
                                np.where(df_hs["product_name"].str.contains('16'), "苹果16",
                                        np.where(df_hs['product_name'].str.contains('15'), "苹果15",
                                                np.where(df_hs['product_name'].str.contains('14'), "苹果14", '苹果14以下'))),"非苹果")
df_hs.loc[:,"残值"]=df_hs.apply(getrecanzhi,axis=1)
def ml(df_hs):
    df_hs.loc[:,"总应收"]=df_hs["总租金"]+df_hs["残值"]+df_hs["商品溢价费"]
    df_hs.loc[:,"原始毛利"]=df_hs["买断金额"]-df_hs["采购金额"]+df_hs["商品溢价费"]
    df_hs.loc[:,"残值估算毛利"]=df_hs["总应收"]-df_hs["采购金额"]
    df_hs_group = df_hs.groupby('下单月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '采购金额': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'}).rename(columns={'订单号': '订单数','残值估算毛利': '残值估算毛利均值', '原始毛利': '原始毛利均值'})
    return df_hs_group
# 支付宝订单
df_zfb = df_hs[df_hs.order_type=='ZFB_ORDER']
df_zfb_group = ml(df_zfb)
# 京东订单
df_jd = df_hs[df_hs.order_type=='JD_ORDER']
df_jd_group = ml(df_jd)
# 总体订单
df_hs_group = ml(df_hs)
now_day = datetime.now().strftime('%Y%m%d')
# with pd.ExcelWriter(f'F:/宏生/月份毛利_{now_day}.xlsx', engine='xlsxwriter') as writer:
#     df_zfb_group.to_excel(writer, sheet_name='支付宝订单')
#     df_jd.to_excel(writer, sheet_name='京东订单')

In [10]:
# df_hs.order_type.unique()
df_hs_group

,订单数,总租金,残值,商品溢价费,总应收,采购金额,残值估算毛利均值,买断金额,原始毛利均值
下单月份,,,,,,,,,
2025-05,4413,30894193.32,1.971442e+07,0.0,5.060861e+07,36630536.1,3167.477447,55656824.40,4311.418151
2025-06,5314,35545552.69,2.391412e+07,0.0,5.945967e+07,43333206.0,3034.712566,67410848.80,4530.982838
2025-07,6189,40432861.93,2.919204e+07,0.0,6.962490e+07,51260235.0,2967.307519,78692582.95,4432.436250
2025-08,4983,32762107.35,2.476535e+07,0.0,5.752746e+07,42327410.0,3050.381569,63886030.00,4326.433875


In [11]:
# 追加到excel末尾

with pd.ExcelWriter(f'F:/宏生/月份毛利_20250805.xlsx', engine='openpyxl', mode='a') as writer:
    df_hs_group.to_excel(writer, sheet_name='Sheet')


# 归还逾期

In [ ]:
# tor.overdue_type = 2归还逾期
sql = '''select
om.order_number, om.status, om.has_actual, om.relet_status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_overdue_record tor on tor.order_id=om.id
where DATE_FORMAT(om.create_time, '%Y-%m') = '2024-07'
and tor.overdue_type = 2
'''

df_gy = clean.query(sql)

In [ ]:
# path = 'F:/需求/归还逾期/归还逾期.xlsx'
# 订单状态为已完成且买断及为已买断
df_gy.loc[:,'订单状态'] = np.where((df_gy.status2=='已完成')&(df_gy.has_actual==1), '已买断', df_gy.status2)
# 剔除status2为已完成的订单
df_gy_new = df_gy[~(df_gy.status2=='已完成')]
df_gy_new#.to_excel('F:/需求/归还逾期/归还逾期.xlsx', engine='xlsxwriter')

In [ ]:
# 将列名改为中文
df_gy_new#.columns = ['订单号', '订单状态', '是否买断', '归还状态', '订单状态2']
# df_gy.status2.value_counts()
df_gy_new

In [ ]:
df_gy
# 每一天（下单日期）每个渠道的退货订单数，


# 倩姐需求

## 每个月的部分免押和非免押的订单数

In [ ]:
sql = '''
select  om.create_time, om.id as order_id, top.total_freeze_fund_amount, om.order_number
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_order om
left join db_digua_business.t_order_pay top on om.id=top.order_id
where top.pay_type='ZFBYSQ'
and top.`status` in (2,5)
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-01-01'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-31 '
'''
df_order = clean.query(sql)



In [ ]:
df = df_order.copy()
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')

df['年份'] = df["下单日期"].dt.year
# 设置免押
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")

# 去重订单数
df=df.drop_duplicates(subset=['order_id'])

In [ ]:
# dfck = df[df['是否出库']==1]
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(df["下单月份"],df["押金类型"],margins=True)
# 写入excel
# df_my.to_excel('F:/需求/倩姐需求/免押数据.xlsx', engine='xlsxwriter')
df_my

## 按月分组的业务量数据

In [58]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` , om.create_time 订单创建时间
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  status2
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, tod.new_actual_money 
-- , tprm.purchase_amount, tprm.rent_received, tprm.all_rental

-- 2025.4.29
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
where DATE_FORMAT(om.create_time, '%Y-%m-%d') <='2025-07-31'
;
'''

df_zhys = clean.query(sql)
# (27787, 62)
df_zhys.shape

(60634, 73)

In [59]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="星晟数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="蘑菇时间"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
# df_zhys.订单创建时间.unique()

In [60]:
# 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# 未完成订单的签约价-已收new_actual_money-rent_received
# 剔除商家拒量的订单
df = df_zhys.copy()
df["下单日期"] = df["订单创建时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["订单创建时间"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
# df['hour'] = df['订单创建时间'].dt.hour
df.下单月份.unique()

array(['2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
       '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
       '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
       '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
       '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
       '2025-07'], dtype=object)

In [72]:
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df['归属渠道'] = np.where(df.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', df.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
df['归属渠道'] = np.where(df.归属渠道.isin(conditions_1), 'S量', df.归属渠道)
df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
df['是否号卡'] = np.where(df.type == 4, 1, 0)

# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
df = df[df.是否出库==1]

# 订单去重
dict_status_code = {
            "订单取消": 1,
            "待支付": 2,
            "已退款": 3,
            "待审核": 4,
            "待发货": 5,
            "待收货": 6,
            "租赁中": 7,
            "已完成": 8
        }
df["状态编码"] = df["status2"].map(dict_status_code)
df.sort_values(by=["下单日期", "状态编码"], inplace=True)
# 删除重复单号
df.drop_duplicates(subset=["order_id"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
# 去刷单订单
# df.drop(df[df['total_describes'].str.contains(pat='panli', regex=False) == True].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df.drop_duplicates(subset=["order_id"], inplace=True)
df.drop_duplicates(subset=["true_name", "mobile", "id_card_num", "下单日期"], keep="last", inplace=True)
df.drop(df[df['true_name'].isin(
        ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
        , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)


# 依据订单编号查重，如有重复输出重复的订单记录
duplicate=df[df.duplicated(subset=['order_number'], keep=False)]


duplicate


,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,status,订单创建时间,status2,订单完成时间,order_method,true_name,id_card_num,mobile,product_name,new_actual_money,channel_name_cc,channel_type_id,activity_name,type,order_type,tips,is_vip,status_result,下单日期,月份,下单月份,年份,进件,是否进件,是否出库,来源渠道,归属渠道,状态编码


In [73]:


df_copy = df[['all_rental', 'purchase_amount' ,'new_actual_money', 'rent_received','status']]
df_copy.loc[:,'剩余应收租金' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['all_rental']-df_copy['rent_received'], 0)
df_copy.loc[:,'剩余总应收' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['new_actual_money']-df_copy['rent_received'], 0)
# 将每一列所有金额汇总
df_copy.rename(columns={'all_rental':'总租金','purchase_amount': '采购金额','new_actual_money': '签约价', 'rent_received': '实收租金'}, inplace=True)

In [74]:

data_res = df_copy.sum()
# 将数值单位由个转万元
data_res['应收总租金（万元）'] = (data_res['总租金']/10000).round(2)
data_res['采购金额（万元）'] = (data_res['采购金额']/10000).round(2)
data_res['签约价（万元）'] = (data_res['签约价']/10000).round(2)
data_res['实收租金（万元）'] = (data_res['实收租金']/10000).round(2)
data_res['剩余总应收'] = (data_res['剩余总应收']/10000).round(2)
data_res['剩余应收租金（万元）'] = (data_res['剩余应收租金']/10000).round(2)
# 写入excel
data_res[['应收总租金（万元）', '采购金额（万元）', '签约价（万元）', '实收租金（万元）', '剩余总应收','剩余应收租金（万元）']].T#.to_excel('F:/需求/倩姐需求/租金数据.xlsx', sheet_name='租金数据')

应收总租金（万元）     36165.59
采购金额（万元）      43895.88
签约价（万元）       67384.04
实收租金（万元）      22593.35
剩余总应收         40291.13
剩余应收租金（万元）    13852.36
dtype: float64

In [8]:
df_copy.下单月份.unique()

array(['2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
       '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
       '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
       '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
       '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
       '2025-07', '2025-08'], dtype=object)

In [33]:
# 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['new_actual_money']-df_copy['rent_received'], 0)
# 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
df_res = df_copy[df_copy['purchase_amount']>0].groupby('下单月份').agg({'new_actual_money': 'sum','rent_received': 'sum', '应收未收金额': 'sum'}).rename(columns={'new_actual_money': '应收金额','rent_received': '已收金额'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# 写入excel
#with pd.ExcelWriter(f'F:/需求/倩姐需求/应收未收金额数据.xlsx', engine='openpyxl') as writer:
    #df_res.to_excel(writer, sheet_name='应收未收金额数据')
# 求和应收未收列
# df_res['应收未收金额'].sum()
# df_res

KeyError: 'new_actual_money'

In [25]:
df_res['应收未收金额'].sum()

np.float64(410561778.66999996)

# 临时需求

In [33]:
sql = """
select om.order_number
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,tolog.status 物流状态, 
from db_digua_business.t_order_jd_out_no tojo
left join db_digua_business.t_order om on tojo.order_id = om.id
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
where date_format(tojo.create_time, '%Y-%m-%d')>='2025-07-01'
and date_format(tojo.create_time, '%Y-%m-%d')<='2025-08-05'
"""
df_jd = clean.query(sql)

In [34]:
df_jd['status2'].value_counts()

status2
已退款     37804
订单取消     2829
租赁中      1541
待收货        94
待审核        49
已完成        23
待发货         9
Name: count, dtype: int64

In [35]:
df_jd['物流状态'].value_counts()

物流状态
5.0    18
Name: count, dtype: int64

In [31]:
sql_jd_md = '''
    select distinct o.order_number, od.new_actual_money 买断价, tojr.create_time,tolog.status 物流状态
    
    from db_digua_business.t_order as o 
    left join db_digua_business.t_order_details as od on o.id = od.order_id 
    left join db_digua_business.t_order_jd_request tojr on tojr.order_number=o.order_number
    left join db_digua_business.t_order_logistics tolog on tolog.order_id=o.id
    where date_format(tojr.create_time, '%Y-%m-%d')>='2025-07-01'
    and date_format(tojr.create_time, '%Y-%m-%d')<='2025-08-05'
    and tojr.request_json like '%IN_THE_LEASE%';
    '''
df_jd_md = clean.query(sql_jd_md)
df_jd_md.head()

,order_number,买断价,create_time,物流状态
0,A20250624161739324,12898.7,2025-07-01 09:00:10,NaN
1,A20250626142459489,12898.7,2025-07-01 09:00:11,NaN
2,A20250627003458104,12898.7,2025-07-01 09:00:12,NaN
3,A20250629110945117,12898.7,2025-07-01 09:00:32,NaN
4,A20250624134038536,11608.7,2025-07-01 12:00:09,NaN


In [32]:
df_jd_md['物流状态'].value_counts()

物流状态
5.0    8
Name: count, dtype: int64

In [41]:
df_jd_md = df_jd_md.sort_values(by='create_time', ascending=False).groupby('order_number').head(1)
df_jd_md.loc[:, '同步日期'] = df_jd_md.create_time.dt.strftime('%Y-%m-%d')
df_jd_md.loc[:, '买断价均值'] = df_jd_md.买断价
df_jd_md.loc[:, '合计买断价'] = df_jd_md.买断价
df_jd_md_group = df_jd_md.groupby('同步日期').agg({'买断价均值': 'mean', '合计买断价': 'sum'})
df_jd_md_group.合计买断价.sum()
# np.float64(17514333.7)

np.float64(20121704.3)

## select

In [77]:
def select_data():
    sql1 = f''' -- 订单&风控信息  近10日数据   
            SELECT om.create_time,om.id as order_id ,om.order_number,om.status, date(om.create_time) as create_date
            ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
            when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
            when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
            ,tod.sku_attributes,tod.product_name,tod.new_actual_money
            ,om.user_mobile,tmu.true_name,tmu.id_card_num
            ,top.total_describes
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
            ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
            ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
            ,cc.name as channel_name         -- 来源渠道
            ,cc.channel_type_id              -- 渠道id
            ,pa.name as activity_name        -- 活动名称
            ,om.merchant_id,om.merchant_name,pa.type
            ,om.order_method, om.activity_id
            ,om.order_type, tor.update_time, tomt.reason, tpmn.name 机型, cc.scene, tp.id 商品ID, tp.name phone_name
            ,tolog.status 物流状态, tolog.sign_time 签收时间
            from  db_digua_business.t_order  om
            left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
            left join db_digua_business.t_order_risk tor on om.id = tor.order_id
            -- 备注信息合并 
            left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
            on om.id = top.order_id 
            -- 渠道名称
            left join db_digua_business.t_channel cc on om.channel = cc.scene 
            -- 活动名称
            left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
            -- 用户信息 
            left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
            -- 商品信息
            left join db_digua_business.t_order_details tod on om.id = tod.order_id
            -- 商家订单转移表
            left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
            -- 商品表
            left join db_digua_business.t_product tp on tp.id = tod.product_id
            -- 商品型号
            left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
            -- 物流表
            left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
            where om.user_mobile is not null 
            and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
            ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
            and date_format(om.create_time, '%Y-%m-%d')>='2025-07-01'
            and date_format(om.create_time, '%Y-%m-%d')<='2025-07-31'
                
            '''
    df_order = clean.query(sql1)
    df_order = df_order[df_order.type != 4]
    sql_risk = ''' -- risk等级
                    select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
                    from db_credit.risk
                    '''
    df_risk = clean.query(sql_risk)
    sql3 = '''
                SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
                '''
    df_risk_examine = clean.query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

    sql_ra = ''' -- 996强拒表
                select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
                '''
    df_ra = clean.query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

    sql_upv = '''
    select
    tsc.scene, date(tsc.day) c_date, tsc.new_uv_count, tsc.uv_count, tsc.pv_count
    from db_digua_business.t_statistics_channel tsc
    '''
    df_upv = clean.query(sql_upv)
    df_order.loc[:, 'phone_name'] = df_order.phone_name.str.replace(' ', '').str.extract(r'(iPhone\d+(ProMax|Pro|Plus)?)')[0]
    df_order = df_order.merge(df_upv, left_on=['scene', 'create_date'], right_on=['scene', 'c_date'], how='left')
    df_order.loc[:,'商品ID'] = df_order.商品ID.astype(str)+'_'+df_order.phone_name


    return df_order, df_risk, df_risk_examine, df_re, df_ra


## clean

In [79]:
def clean_data(df, df_risk, df_re, df_ra, key=None):
    # 日期处理
    df["下单日期"] = df["create_time"].dt.date
    df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
    df["月份"] = df["下单日期"].dt.month
    df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    # 备注信息处理
    df['拒绝理由'] = df["rejected"].str.replace("[", "").str.replace("]", "").str.replace('"', '')
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"] = df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"] = df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"] = df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()

    df.loc[:, "内存"] = df.apply(lambda x: clean.getneicun(x["sku_attributes"]), axis=1)
    df.loc[:, '机型内存'] = df.机型+'_'+df.内存
    df.loc[:, "进件"] = np.where((df["status2"] == "待支付") | (df["status2"] == "订单取消"), "未进件", "进件")
    df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
    # 渠道归属
    df.loc[:, "归属渠道"] = df.apply(
        lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"],
                                        x['channel_type_id'], x['order_type']), axis=1)
    # 京东不去重数据，不需要执行订单去重处理
    if key != '京东':
        # 订单去重处理
        df = clean.order_drop_duplicates(df)

    # 定义状态处理
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(
        df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(
        df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'],
        right_on=['order_id', 'time_ra'], how='left')
    df['是否前置拦截'] = np.where(
        (df.result.str.contains('id_card不得为空')) | (df.result.str.contains('mobile校验不通过')) | (
            df.result.str.contains('name校验不通过'))
        | (df.result.str.contains('年龄超过49岁或低于18岁')) | (df.result == '风控未通过') | (
                df.result == '黑名单用户'), 1, 0)
    df['是否机审强拒'] = np.where(df.status_r == '1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re == '1') | (df.status_ra == '1'), 1, 0)
    df.loc[:, "审核状态"] = df.apply(
        lambda x: clean.reject_type(x["拒绝理由"], x["进件"], x["电审拒绝原因"], x["取消原因"], x["status2"],
                                        x["无法联系原因"], x["total_describes"], x['是否前置拦截'], x['是否机审强拒'],x['是否出库前风控强拒']), axis=1)
    # 剔除商家数据
    df = clean.drop_merchant(df)
    # 获取节点状态数据
    df = clean.status_node(df)

    # 剔除拒收数据
    # df = df[df['物流状态'] != 5]
    # 剔除据量数据
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量 == 0]



    return df, df2

## run

In [80]:
def run():
    df, df_risk, df_risk_examine, df_re, df_ra = select_data()

    return df, df_risk, df_risk_examine, df_re, df_ra

In [81]:
# def jd_job(self, hour, minute, path, hours):
date = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')# 15
today = datetime.now().strftime('%Y%m%d')
# print(f'执行定时任务：现在是{today}的{hour}点{minute}分...')
print('正在查询数据...')
# 如果需要更改时间，还需要改这儿date_add(current_date, interval - 31 day)
sql_jd = ''' -- 京东外部订单关联表
            select  tojo.create_time, tojo.order_id, tojo.status from db_digua_business.t_order_jd_out_no tojo
            where 
            date_format(tojo.create_time, '%Y-%m-%d')>='2025-07-01'
            and date_format(tojo.create_time, '%Y-%m-%d')<='2025-07-31'
            -- tojo.create_time>=date_add(current_date, interval - 15 day) and tojo.create_time<current_date
        '''
df_jd = clean.query(sql_jd)

df_jd.loc[:, '下单日期'] = pd.to_datetime(df_jd.create_time.dt.strftime('%Y-%m-%d'))
# print(df_jd['下单日期'].unique())
df, df_risk, df_risk_examine, df_re, df_ra = run()
print('数据查询完毕！\n正在清理数据...')
df, df2 = clean_data(df, df_risk, df_re, df_ra, key='京东')
print('数据清理完毕！\n正在计算数据...')
# 关联数据
df_jd_new = df_jd.merge(df[['order_id', 'order_number', 'new_actual_money', 'status2','签收时间']], on='order_id', how='left')
# 区分京东渠道并计算进件和风控通过的件数
df = df[df.归属渠道=='京东渠道']
df2 = df2[df2.归属渠道=='京东渠道']
df_group = all_models.data_group(df, df2, df_risk_examine, '下单日期')


# 获取京东的不去重买断价
# 如果需要更改时间，还需要改这儿date_add(current_date, interval - 31 day)
# tojr.request_json like '%IN_THE_LEASE%'表已签收，不含拒收


正在查询数据...
数据查询完毕！
正在清理数据...
数据清理完毕！
正在计算数据...


In [108]:
sql_jd_md = '''
select distinct o.order_number, od.new_actual_money 买断价, tojr.create_time from db_digua_business.t_order as o 
left join db_digua_business.t_order_details as od on o.id = od.order_id 
left join db_digua_business.t_order_jd_request tojr on tojr.order_number=o.order_number
where 
date_format(tojr.create_time, '%Y-%m-%d')>='2025-07-01'
and date_format(tojr.create_time, '%Y-%m-%d')<='2025-07-31'
-- tojr.create_time>=date_add(current_date, interval -15 day)
and tojr.request_json like '%IN_THE_LEASE%'
'''
df_jd_md = clean.query(sql_jd_md)
df_jd_md = df_jd_md.sort_values(by='create_time', ascending=False).groupby('order_number').head(1)
df_jd_md.loc[:, '同步日期'] = df_jd_md.create_time.dt.strftime('%Y-%m-%d')
df_jd_md.loc[:, '买断价均值'] = df_jd_md.买断价
df_jd_md.loc[:, '合计买断价'] = df_jd_md.买断价
df_jd_md_group = df_jd_md.groupby('同步日期').agg({'买断价均值': 'mean', '合计买断价': 'sum'})
# 计算京东的去重，待发货，已发货和买断价数据


In [143]:
df_jd_md#.to_excel(f'F:/需求/瑞彤需求/京东订单物流买断_1.xlsx', index=False)

,order_number,买断价,create_time,同步日期,合计买断价
1391,A20250729163416651,12898.7,2025-07-31 18:00:44,2025-07-31,12898.7
1390,A20250729150602133,12898.7,2025-07-31 18:00:40,2025-07-31,12898.7
1389,A20250728224200205,12898.7,2025-07-31 18:00:33,2025-07-31,12898.7
1388,A20250728172803495,12898.7,2025-07-31 18:00:29,2025-07-31,12898.7
1387,A20250728162956612,12898.7,2025-07-31 18:00:28,2025-07-31,12898.7
...,...,...,...,...,...
5,A20250624213326180,12898.7,2025-07-01 12:00:09,2025-07-01,12898.7
3,A20250629110945117,12898.7,2025-07-01 09:00:32,2025-07-01,12898.7
2,A20250627003458104,12898.7,2025-07-01 09:00:12,2025-07-01,12898.7
1,A20250626142459489,12898.7,2025-07-01 09:00:11,2025-07-01,12898.7


In [115]:
# 将df_jd_md与df2左连接
df_jd_md_merge = df_jd_md.merge(df_jd_new[['订单号','下单日期','订单id', '订单状态']], left_on='order_number',right_on='订单号', how='left')

In [103]:

# df_jd_md_merge.to_excel(f'F:/需求/瑞彤需求/京东订单物流买断', index=False)

In [61]:
# df_jd_new.loc[:, '待发货'] = np.where(df_jd_new.status=='TO_SEND_GOODS', 1, 0)
df_jd_new.loc[:, '待发货'] = np.where((df_jd_new.status=='TO_SEND_GOODS')& (df_jd_new.status2=='待发货'), 1, 0)
df_jd_new.loc[:, '已发货'] = np.where(df_jd_new.status.isin(['IN_DELIVERY', 'IN_THE_LEASE']), 1, 0)
df_jd_new_group = df_jd_new.groupby('下单日期').agg({'order_id': 'count', '待发货': 'sum', '已发货': 'sum'}).rename(columns={'order_id': '创建订单数'})
# # 关联各个节点的数据
df_jd_group = df_jd_new_group.merge(df_group[['进件数', '风控通过件']], on='下单日期', how='left')
df_jd_group.loc[:, '审核通过率'] = (df_jd_group.风控通过件/df_jd_group.创建订单数).map(lambda x: format(x, '.2%'))
df_jd_group.loc[:, '风控通过率'] = (df_jd_group.风控通过件/df_jd_group.进件数).map(lambda x: format(x, '.2%'))
df_jd_group.loc[:, '发货率'] = (df_jd_group.已发货/df_jd_group.创建订单数).map(lambda x: format(x, '.2%'))
df_jd_group = df_jd_group[['创建订单数', '进件数', '风控通过件', '审核通过率', '风控通过率', '待发货', '已发货', '发货率']]
df_jd_group.index = df_jd_group.index.astype(str)
df_jd_group = pd.concat([df_jd_group, df_jd_md_group], axis=1)

print('数据计算完毕！\n正在写入数据...')

数据计算完毕！
正在写入数据...


## 买断优召

In [140]:
sql_lzy = f'''
    select
    ybt.order_id 订单ID, tod.new_actual_money 买断价, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 实收买断金额,
    tprm.purchase_amount 采购金额, ybt.`status`, ybt.follow_log_tag, ybt.type, om.has_actual, date_format(ybt.end_time, '%Y-%m') 月份
    from db_rent.ya_buyout_task ybt
    left join (select order_id, discount_money from db_digua_business.t_order_out where discount_money>0) too on too.order_id=ybt.order_id
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=ybt.order_id
    left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
    left join db_digua_business.t_order om on om.id=ybt.order_id
    where date_format(ybt.end_time, '%Y-%m')='2025-07'
    
    '''

df_lzy = clean.query(sql_lzy)

In [142]:
df_lzy[df_lzy['订单ID'] == 2881400]

,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1599,2881400,15596.3,NaN,NaN,10350.0,3,0,1,1,2025-07


In [138]:
df_lzy_tq = df_lzy[(df_lzy.status.isin([3, 4])) & (df_lzy.follow_log_tag != 4) & (df_lzy.type == 1) & (df_lzy.has_actual == 1)]
# 填充空值为0
# df_lzy_tq.fillna(0, inplace=True)
df_lzy_tq

,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1257,1697636,12996.7,400.0,12596.7,9700.0,3,0,1,1,2025-07
1434,2290668,12996.7,1200.0,11796.7,8810.0,4,0,1,1,2025-07
1507,2733468,12996.7,1200.0,11796.7,8240.0,3,0,1,1,2025-07
1508,1886094,10397.1,600.0,9797.1,7650.0,4,0,1,1,2025-07
1518,2350912,7797.5,200.0,7597.5,5719.0,4,0,1,1,2025-07
1528,1879510,12996.7,500.0,12496.7,9650.0,3,0,1,1,2025-07
1534,2188686,12996.7,800.0,12196.7,8910.0,4,0,1,1,2025-07
1535,2188686,12996.7,800.0,12196.7,8910.0,3,0,1,1,2025-07
1541,2389809,12997.7,1000.0,11997.7,8770.0,3,0,1,1,2025-07
1549,1275873,6719.0,0.0,6719.0,4930.0,3,0,1,1,2025-07


In [139]:
# 优惠金额填充空值为0,实收买断金额填充空值为买断价
df_lzy_tq.loc[:, '优惠金额'] = df_lzy_tq.优惠金额.fillna(0)
# # 优惠金额填充空值为0,实收买断金额填充空值为买断价
df_lzy_tq.loc[:, '实收买断金额'] = df_lzy_tq.实收买断金额.fillna(df_lzy_tq.买断价)

df_lzy_tq


,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1257,1697636,12996.7,400.0,12596.7,9700.0,3,0,1,1,2025-07
1434,2290668,12996.7,1200.0,11796.7,8810.0,4,0,1,1,2025-07
1507,2733468,12996.7,1200.0,11796.7,8240.0,3,0,1,1,2025-07
1508,1886094,10397.1,600.0,9797.1,7650.0,4,0,1,1,2025-07
1518,2350912,7797.5,200.0,7597.5,5719.0,4,0,1,1,2025-07
1528,1879510,12996.7,500.0,12496.7,9650.0,3,0,1,1,2025-07
1534,2188686,12996.7,800.0,12196.7,8910.0,4,0,1,1,2025-07
1535,2188686,12996.7,800.0,12196.7,8910.0,3,0,1,1,2025-07
1541,2389809,12997.7,1000.0,11997.7,8770.0,3,0,1,1,2025-07
1549,1275873,6719.0,0.0,6719.0,4930.0,3,0,1,1,2025-07


# 发货率

In [46]:
sql1 = '''    
        select o.order_number,od.product_id,od.product_name,o.merchant_id,o.merchant_name,o.`status`,o.create_time,op.pay_date,ol.go_express_date,
        o.alipay_order_id,cc.`name` as channel_name,pa.name as activity_name,o.order_method,cc.channel_type_id, o.order_type, o.day, o.has_order_check, pa.type,
        tpr.min_create_time, op.status as status_o
        from db_digua_business.t_order o 
        left join db_digua_business.t_order_pay op on op.order_id = o.id and op.pay_type = 'ZFBYSQ' and op.sync_mini_order = 'Y'
        left join db_digua_business.t_order_logistics ol on o.id = ol.order_id 
        left join db_digua_business.t_order_details od on od.order_id = o.id
        -- 渠道名称
        left join db_digua_business.t_channel cc on o.channel = cc.scene 
        -- 活动名称
        left join db_digua_business.t_platform_activity pa on o.activity_id = pa.id
        LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = o.merchant_id
        left join (SELECT tpr.order_id, min(tpr.create_time) as min_create_time FROM db_digua_business.t_pay_record tpr GROUP BY tpr.order_id) tpr on tpr.order_id=o.id
        where o.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day ) 
        -- and tmer.shop_type!=2
        and o.user_mobile is not null 
        '''

df_order = clean.query(sql1)
df_order.shape

(382534, 20)

In [47]:
df = df_order.copy()

df["下单日期"] = pd.to_datetime(df["create_time"]).dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
df["月份"] = df["下单日期"].dt.month
df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
df.loc[:, "归属渠道"] = df.apply(lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df.drop_duplicates(subset=["order_number"], inplace=True)
# df = df[df.status_o.isin([2, 5])]# , 5
# df = df.reset_index(drop=True)

df.shape

(74325, 24)

In [ ]:
# 原回捞逻辑订单数
# df[df.order_id == 3538653]
df_2412 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
df_2412.shape# 1839
df_2412.loc[:, '机审强拒_拒量'] = np.where(df_2412.qvt_risk=='1', 1, 0)
df_2412[df_2412.机审强拒_拒量==0].head()#.shape


,create_date,create_time,order_id,order_number,all_money,status,user_id,status2,channel_type,sku_attributes,product_name,new_actual_money,product_id,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,dy_order_item_json,type,buy_service_product,service_status,order_method,activity_id,appid,max_overdue_days,update_time,reason,purchase_amount,违约金,order_type,union_rent_tag,app_type,已收租金,物流状态,rejected_ato_risk,id,classify_id,商品ID,phone_name,下单日期,月份,下单月份,年份,hour,拒绝理由,取消原因,电审拒绝原因,无法联系原因,商品类型,租赁方案,押金类型,优惠券使用否,是否进行预授权,颜色,内存,进件,来源渠道,退货拒签,归属渠道,状态编码,status_r,status_re,time_ra,status_ra,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,下单时段,待审核,前置拦截,人审拒绝,客户取消,无法联系,是否进件,是否出库,进件前取消,出库前风控强拒,机审强拒,机审通过件,风控通过件,已退款,是否二手,是否拒量,机审强拒_拒量
21491,2025-08-15,2025-08-15 11:36:23,3626824,A20250815113622624,15596.30,10,25076624,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,15596.3,6691,租完即送,18718648714,李伟权,440902199509193218,"[""后置风控拒绝""]","{""code"":0,""gps_id_card"":""{\""3日内GPS(1000)米内下单人数...",后置风控拒绝,661e7a7bd39204383a0650d7d01d4ab9,"[命中极信sc32007分强拒, 命中策略strategy_240801强拒]",【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。,"[用户下单次数:1, 3日内GPS(1000)米内下单人数:2, 7日内GPS(1000)米...",None,None,None,1,顶部搜索框的搜索结果页,48,端内价格-全新苹果,14,维客自营账户,0.0,None,6.0,1,NaN,0,180,2021003131606511,NaN,2025-08-15 14:17:14,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,6691,116.0,6691_iPhone16ProMax,iPhone16ProMax,2025-08-15,8,2025-08,2025,11,"命中极信sc32007分强拒, 命中策略strategy_240801强拒",NaN,NaN,NaN,全新,租完即送,全免押,未使用,已预授权,白色钛金属,512GB,进件,顶部搜索框的搜索结果页,0,搜索渠道,3.0,1,NaN,NaT,NaN,0,1,0,机审强拒,2025-08-15 11:,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
21515,2025-08-15,2025-08-15 11:44:08,3626965,A20250815114408771,7331.88,10,24507000,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,12996.7,6691,租物归还,13859123379,严倩倩,350181198905171948,"[""88888_廖志彧审核订单A20250815114408771，审核结果：不通过,msg...","{""code"":0,""grade"":""A"",""gradeMsg"":""【优质订单】免人审"",""...","88888_廖志彧审核订单A20250815114408771，审核结果：不通过,msg=审...",a0415b1b5dde36f34d39627501ecb852,[],【风控通过】该用户风控资质较好，建议商户可以优先审核发货。为保证订单流程规范，发货前请注意核...,"[回捞策略241205命中(1), 已有在租, 用户下单次数:2, 用户下单次数:2]",None,None,None,1,顶部搜索框的搜索结果页,48,端内价格-全新苹果,14,维客自营账户,0.0,None,6.0,1,NaN,0,180,2021003131606511,NaN,2025-08-15 14:11:23,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,6691,116.0,6691_iPhone16ProMax,iPhone16ProMax,2025-08-15,8,2025-08,2025,11,,NaN,高风险行业人员,NaN,全新,租完归还,全免押,未使用,已预授权,黑色钛金属,256GB,进件,顶部搜索框的搜索结果页,0,搜索渠道,3.0,0,NaN,NaT,NaN,0,0,0,人审拒绝,2025-08-15 11:,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0
21531,2025-08-15,2025-08-15 11:47:47,3627054,A20250815114747862,5359.23,10,25076392,已退款,其他渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月5折 华为Pura80 Pro 全新非监管机 一英寸主摄 顺丰包邮【直,9097.3,8220,租物归还,18733328005,王建,130203199212024515,"[""后置风控拒绝""]","{""code"":0,""grade"":""A"",""gradeMsg"":""风控不通过"",""is_v...",后置风控拒绝,9c77a06d4edddfad28796bc63f1615d0,"[命中极信sc32007分强拒, 命中策略strategy_240801强拒]",【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。,"[用户下单次数:1, 用户下单次数:1, 已有在租, 强制转租送, 回捞策略241205命中...",None,None,None,1,支付宝直播,76,支付宝直播商品2,14,维客自营账户,0.0,None,2.0,1,NaN,0,423,2021003131606511,NaN,2025-08-15 13:46:22,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,8220,116.0,8220_HUAWEI Pura 80 Pro,HUAWEI Pura 80 Pro,2025-08-15,8,2025-08,2025,11,"命中极信sc32007分强拒, 命中策略strategy_240801强拒",NaN,NaN,NaN,全新,租完归还,全免押,未使用,已预授权,釉白,12GB+512GB,进件,支付宝直播,0,支付宝直播,3.0,1,NaN,NaT,NaN,0,1,0,机审强拒,2025-08-15 11:,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
63070,2025-08-21,2025-08-21 11:10:19,3699934,A20250821111018167,7331.88,10,25111133,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,12996.7,6691,租物归还,18645533670,杨亮,232332200211301216,"[""88888_蔡诗静订单A20250821111018167，msg=不接受监管设备"", ...","{""code"":0,""grade"":""A"",""gradeM

In [ ]:
# 原回捞逻辑订单数
# df[df.order_id == 3538653]
df_2412 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
df_2412.shape# 1839
df_2412.loc[:, '机审强拒_拒量'] = np.where(df_2412.qvt_risk=='1', 1, 0)
df_2412[df_2412.机审强拒_拒量==0].head()#.shape


,create_date,create_time,order_id,order_number,all_money,status,user_id,status2,channel_type,sku_attributes,product_name,new_actual_money,product_id,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,dy_order_item_json,type,buy_service_product,service_status,order_method,activity_id,appid,max_overdue_days,update_time,reason,purchase_amount,违约金,order_type,union_rent_tag,app_type,已收租金,物流状态,rejected_ato_risk,id,classify_id,商品ID,phone_name,下单日期,月份,下单月份,年份,hour,拒绝理由,取消原因,电审拒绝原因,无法联系原因,商品类型,租赁方案,押金类型,优惠券使用否,是否进行预授权,颜色,内存,进件,来源渠道,退货拒签,归属渠道,状态编码,status_r,status_re,time_ra,status_ra,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,下单时段,待审核,前置拦截,人审拒绝,客户取消,无法联系,是否进件,是否出库,进件前取消,出库前风控强拒,机审强拒,机审通过件,风控通过件,已退款,是否二手,是否拒量,机审强拒_拒量
21491,2025-08-15,2025-08-15 11:36:23,3626824,A20250815113622624,15596.30,10,25076624,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,15596.3,6691,租完即送,18718648714,李伟权,440902199509193218,"[""后置风控拒绝""]","{""code"":0,""gps_id_card"":""{\""3日内GPS(1000)米内下单人数...",后置风控拒绝,661e7a7bd39204383a0650d7d01d4ab9,"[命中极信sc32007分强拒, 命中策略strategy_240801强拒]",【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。,"[用户下单次数:1, 3日内GPS(1000)米内下单人数:2, 7日内GPS(1000)米...",None,None,None,1,顶部搜索框的搜索结果页,48,端内价格-全新苹果,14,维客自营账户,0.0,None,6.0,1,NaN,0,180,2021003131606511,NaN,2025-08-15 14:17:14,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,6691,116.0,6691_iPhone16ProMax,iPhone16ProMax,2025-08-15,8,2025-08,2025,11,"命中极信sc32007分强拒, 命中策略strategy_240801强拒",NaN,NaN,NaN,全新,租完即送,全免押,未使用,已预授权,白色钛金属,512GB,进件,顶部搜索框的搜索结果页,0,搜索渠道,3.0,1,NaN,NaT,NaN,0,1,0,机审强拒,2025-08-15 11:,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
21515,2025-08-15,2025-08-15 11:44:08,3626965,A20250815114408771,7331.88,10,24507000,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,12996.7,6691,租物归还,13859123379,严倩倩,350181198905171948,"[""88888_廖志彧审核订单A20250815114408771，审核结果：不通过,msg...","{""code"":0,""grade"":""A"",""gradeMsg"":""【优质订单】免人审"",""...","88888_廖志彧审核订单A20250815114408771，审核结果：不通过,msg=审...",a0415b1b5dde36f34d39627501ecb852,[],【风控通过】该用户风控资质较好，建议商户可以优先审核发货。为保证订单流程规范，发货前请注意核...,"[回捞策略241205命中(1), 已有在租, 用户下单次数:2, 用户下单次数:2]",None,None,None,1,顶部搜索框的搜索结果页,48,端内价格-全新苹果,14,维客自营账户,0.0,None,6.0,1,NaN,0,180,2021003131606511,NaN,2025-08-15 14:11:23,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,6691,116.0,6691_iPhone16ProMax,iPhone16ProMax,2025-08-15,8,2025-08,2025,11,,NaN,高风险行业人员,NaN,全新,租完归还,全免押,未使用,已预授权,黑色钛金属,256GB,进件,顶部搜索框的搜索结果页,0,搜索渠道,3.0,0,NaN,NaT,NaN,0,0,0,人审拒绝,2025-08-15 11:,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0
21531,2025-08-15,2025-08-15 11:47:47,3627054,A20250815114747862,5359.23,10,25076392,已退款,其他渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",首月5折 华为Pura80 Pro 全新非监管机 一英寸主摄 顺丰包邮【直,9097.3,8220,租物归还,18733328005,王建,130203199212024515,"[""后置风控拒绝""]","{""code"":0,""grade"":""A"",""gradeMsg"":""风控不通过"",""is_v...",后置风控拒绝,9c77a06d4edddfad28796bc63f1615d0,"[命中极信sc32007分强拒, 命中策略strategy_240801强拒]",【风控拒绝】风控不通过，租户风险较高。建议预收到设备价值或者拒绝。,"[用户下单次数:1, 用户下单次数:1, 已有在租, 强制转租送, 回捞策略241205命中...",None,None,None,1,支付宝直播,76,支付宝直播商品2,14,维客自营账户,0.0,None,2.0,1,NaN,0,423,2021003131606511,NaN,2025-08-15 13:46:22,订单初次分配,NaN,NaN,ZFB_ORDER,N,None,NaN,NaN,None,8220,116.0,8220_HUAWEI Pura 80 Pro,HUAWEI Pura 80 Pro,2025-08-15,8,2025-08,2025,11,"命中极信sc32007分强拒, 命中策略strategy_240801强拒",NaN,NaN,NaN,全新,租完归还,全免押,未使用,已预授权,釉白,12GB+512GB,进件,支付宝直播,0,支付宝直播,3.0,1,NaN,NaT,NaN,0,1,0,机审强拒,2025-08-15 11:,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
63070,2025-08-21,2025-08-21 11:10:19,3699934,A20250821111018167,7331.88,10,25111133,已退款,搜索渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",iPhone 16 Pro Max 非监管机 全新未激活 国行正品 顺丰包邮,12996.7,6691,租物归还,18645533670,杨亮,232332200211301216,"[""88888_蔡诗静订单A20250821111018167，msg=不接受监管设备"", ...","{""code"":0,""grade"":""A"",""gradeM

In [49]:
def xcx( df):
    # 筛选所需要的字段
    # df = df[df.status_o.isin([2, 5])]
    df_all = df[['order_number', 'product_id', 'product_name', 'merchant_id','merchant_name', 'status', 'create_time', 'pay_date', 'go_express_date',
                'alipay_order_id', 'channel_name','下单日期', '月份', '来源渠道', '归属渠道']]
    df_all = df_all.drop_duplicates(subset=['order_number'])
    # 取到日期并对字段进行重命名
    df_all['create_time'] = df_all['create_time'].dt.date
    df_all['pay_date'] = df_all['pay_date'].dt.date
    df_all['go_express_date'] = df_all['go_express_date'].dt.date
    df_all.rename(columns={'order_number': '订单编号', 'product_id': '商品ID', 'product_name': '商品名称','merchant_id': '店铺ID', 'merchant_name': '店铺名称',
                        'status': '订单状态', 'create_time': '订单生成时间', 'pay_date': '支付时间','go_express_date': '发货时间', 'alipay_order_id': '支付宝流水号'}, inplace=True)
    df_all['发货时效'] = (pd.to_datetime(df_all['发货时间']) - pd.to_datetime(df_all['支付时间'])).dt.days
    # 筛选7天发货时效并计算发货率l
    df_all.loc[:, '发货数'] = np.where(pd.to_datetime(df_all['发货时间'])-pd.to_datetime(df_all['支付时间'])<=pd.Timedelta(hours=168), 1, 0)
    df_all_zh = df_all.groupby('订单生成时间').agg({'订单编号': 'size', '发货数': 'sum'})
    df_all_zh.loc['汇总', :] = df_all_zh.sum(axis=0)
    df_all_zh['发货率'] = (df_all_zh['发货数'] / df_all_zh['订单编号'] * 100).apply(lambda x: f'{x: .2f}%')
    df_all_zh.rename(columns={'订单编号': '进件量'}, inplace=True)

    return df_all_zh

In [50]:
def get_data(df):
    # 近30天发货率
    # 芝麻租物
    df_zm = df[df['归属渠道'] == '芝麻租物']
    print(len(df_zm))
    df_zm_zh_30 = xcx(df_zm)
    # 搜索渠道
    df_ss = df[df['归属渠道'] == '搜索渠道']
    print(len(df_ss))
    df_ss_zh_30 = xcx(df_ss)

    # # 近7天发货率
    # # 芝麻租物
    # df_zm_7 = df_zm[df_zm.下单日期>=(datetime.now()-timedelta(days=7))]
    # df_zm_zh_7 = xcx(df_zm_7)
    # # 搜索渠道
    # df_ss_7 = df_ss[df_ss.下单日期 >= (datetime.now() - timedelta(days=7))]
    # df_ss_zh_7 = xcx(df_ss_7)

    # # 48小时发货率 总体
    # df_48h = fh48(df)
    # # 芝麻租物
    # df_zm_48h = fh48(df_zm)
    # # 搜索渠道
    # df_ss_48h = fh48(df_ss)
    return  df_zm_zh_30, df_ss_zh_30#, df_zm_zh_7, df_ss_zh_7, df_48h, df_zm_48h, df_ss_48h

df_zm_zh_30, df_ss_zh_30 = get_data(df)

20882
11162


In [ ]:
df_zm_zh_30#[-6:]

In [ ]:
df_ss_zh_30#[-5:]